In [2]:
import numpy as np
import pandas as pd
import os.path
from sklearn.metrics import roc_auc_score
import tqdm

# Santander Customer Transaction Prediction (KAGGLE)

Leer de que trata la competencia

https://www.kaggle.com/c/santander-customer-transaction-prediction/overview

# Bajada de Dataset
La siguiente celda baja el dataset y lo guarda en la carpeta /data.

Luego de ejecutarla debería tener los archivos train.csv y test.csv en esta carpeta

# Carpeta on dataset:
https://drive.google.com/drive/folders/19scmxZ64fe7mIneDwG21eF_QLn-8-S8j?usp=sharing

In [5]:
#Si quiero habilitar mi google drive descomentar
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Cargar datos de train

In [6]:
df_train = pd.read_csv('/content/gdrive/MyDrive/data/train.csv')
df_test = pd.read_csv('/content/gdrive/MyDrive/data/test.csv')

In [7]:
# Miramos las primeras 5 observaciones del dataset
df_train.head()

# TODO: Importar datos de test y asignarlos a la variable df_test desde la carpeta data/
# df_test  = 

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,29.4846,5.8683,3.8208,15.8348,-5.0121,15.1345,3.2003,9.3192,3.8821,5.7999,5.5378,5.0988,22.0330,5.5134,30.2645,10.4968,-7.2352,16.5721,-7.3477,11.0752,-5.5937,9.4878,-14.9100,9.4245,22.5441,-4.8622,7.6543,-15.9319,13.3175,-0.3566,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,13.2070,5.8442,4.7086,5.7141,-1.0410,20.5092,3.2790,-5.5952,7.3176,5.7690,-7.0927,-3.9116,7.2569,-5.8234,25.6820,10.9202,-0.3104,8.8438,-9.7009,2.4013,-4.2935,9.3908,-13.2648,3.1545,23.0866,-5.3000,5.3745,-6.2660,10.1934,-0.8417,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,31.8833,5.9684,7.2084,3.8899,-11.0882,17.2502,2.5881,-2.7018,0.5641,5.3430,-7.1541,-6.1920,18.2366,11.7134,14.7483,8.1013,11.8771,13.9552,-10.4701,5.6961,-3.7546,8.4117,1.8986,7.2601,-0.4639,-0.0498,7.9336,-12.8279,12.4124,1.8489,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,33.5107,5.6953,5.4663,18.2201,6.5769,21.2607,3.2304,-1.7759,3.1283,5.5518,1.4493,-2.6627,19.8056,2.3705,18.4685,16.3309,-3.3456,13.5261,1.7189,5.1743,-7.6938,9.7685,4.8910,12.2198,11.8503,-7.8931,6.4209,5.9270,16.0201,-0.2829,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [10]:
print(pd.value_counts(df_train.target))

print(179902/(179902+ 20098))

0    179902
1     20098
Name: target, dtype: int64
0.89951


In [27]:
np.unique(np.array(df_train.target))

array([0, 1])

In [11]:
df_train.columns

Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=202)

Cuantas features (covariables, variable explicativa, variable independiente, variables exógenas) hay (dimensión de las observaciones, regresoras)? Que columna contiene la salida (variable dependiente, variable endógena)? Cuantas clases hay? Cuantas observaciones tiene train?

In [12]:
df_train.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,7.567236,0.394340,-3.245596,14.023978,8.530232,7.537606,14.573126,9.333264,-5.696731,15.244013,12.438567,13.290894,17.257883,4.305430,3.019540,10.584400,13.667496,-4.055133,-1.137908,5.532980,5.053874,-7.687740,10.393046,-0.512886,14.774147,11.434250,3.842499,2.187230,5.868899,10.642131,...,24.259300,5.633293,5.362896,11.002170,-2.871906,19.315753,2.963335,-4.151155,4.937124,5.636008,-0.004962,-0.831777,19.817094,-0.677967,20.210677,11.640613,-2.799585,11.882933,-1.014064,2.591444,-2.741666,10.085518,0.719109,8.769088,12.756676,-3.983261,8.970274,-10.335043,15.377174,0.746072,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,1.235070,5.500793,5.970253,0.190059,4.639536,2.247908,0.411711,2.557421,6.712612,7.851370,7.996694,5.876254,8.196564,2.847958,0.526893,3.777245,0.285535,5.922210,1.523714,0.783367,2.615942,7.965198,2.159891,2.587830,4.322325,0.541614,5.179559,3.119978,2.249730,4.278903,...,10.880263,0.217938,1.419612,5.262056,5.457784,5.024182,0.369684,7.798020,3.105986,0.369437,4.424621,5.378008,8.674171,5.966674,7.136427,2.892167,7.513939,2.628895,8.579810,2.798956,5.261243,1.371862,8.963434,4.474924,9.318280,4.725167,3.189759,11.574708,3.944604,0.976348,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,3.970500,-20.731300,-26.095000,13.434600,-6.011100,1.013300,13.076900,0.635100,-33.380200,-10.664200,-12.402500,-5.432200,-10.089000,-5.322500,1.209800,-0.678400,12.720000,-24.243100,-6.166800,2.089600,-4.787200,-34.798400,2.140600,-8.986100,1.508500,9.816900,-16.513600,-8.095100,-1.183400,-6.337100,...,-7.452200,4.852600,0.623100,-6.531700,-19.997700,3.816700,1.851200,-35.969500,-5.250200,4.258800,-14.506000,-22.479300,-11.453300,-22.748700,-2.995300,3.241500,-29.116500,4.952100,-29.273400,-7.856100,-22.037400,5.416500,-26.001100,-4.808200,-18.489700,-22.583300,-3.022300,-47.753600,4.412300,-2.554300,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.26

In [ ]:
# TODO: Inspeccionar test
# Puede ejecutar los métodos que crea necesarios. Recomendamos describe, columns, shape, head

Que columna esta en los datos de train pero no en los de test? Cuantas observaciones de test hay?

# Dividir data en Train y Val

In [43]:
N_train = 160_000
X_train = df_train.drop(columns=['ID_code', 'target']).values[:N_train]
X_val = df_train.drop(columns=['ID_code', 'target']).values[N_train:]
y_train = df_train['target'].values[:N_train]
y_val = df_train['target'].values[N_train:]

def normal(serie): 
  return (serie-np.mean(serie))/np.std(serie)

X_train=normal(df_train.drop(columns=['ID_code', 'target']).values[:N_train])
X_val=normal(df_train.drop(columns=['ID_code', 'target']).values[N_train:])
#y_train=normal( df_train['target'].values[:N_train])
#y_val=normal(df_train['target'].values[N_train:])

In [38]:
X_train

array([[ 0.22332347, -1.40257448,  0.53197068, ...,  0.18586277,
         0.62222821, -0.81325141],
       [ 0.48980152, -1.12948389,  0.73383419, ...,  0.20918775,
         1.19921615, -0.49833315],
       [ 0.19060227, -0.98444267,  0.54981108, ...,  0.15523192,
         0.82318107, -0.65927964],
       ...,
       [ 0.28809323, -0.74405259,  0.12877142, ...,  0.31944846,
         1.31757955,  0.00877299],
       [ 0.13550814, -0.96776128,  0.51894222, ...,  0.27781741,
         0.57519536,  0.60154203],
       [ 0.5119261 , -0.9227153 ,  0.21913243, ...,  0.11652943,
         0.95966428, -0.61376799]])

# Regresión logística

In [44]:
from sklearn.linear_model import LogisticRegression

In [45]:
model = LogisticRegression(solver='lbfgs', max_iter=100)

In [46]:
%time model.fit(X_train, y_train)

CPU times: user 13.6 s, sys: 2.35 s, total: 16 s
Wall time: 8.33 s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

A que se debe el warning no termina de convergence?

In [47]:
acc = model.score(X_train, y_train)
auc = roc_auc_score(y_train, model.predict_proba(X_train)[:,1])
print(auc)
print(acc)

0.8514768336351206
0.912675


In [48]:
print(roc_auc_score(y_val, model.predict_proba(X_val)[:,1]))
print(model.score(X_val, y_val))

0.8532351658168906
0.911975


cuantos parámetros aprendio el modelo?

# Normalizar los datos

In [50]:
means = X_train.mean(axis=0)
stds = X_train.std(axis=0)

In [51]:
X_train_normalized = (X_train - means)/stds
X_val_normalized = (X_val - means)/stds

In [52]:
model_2 = LogisticRegression(solver='lbfgs')
%time model_2.fit(X_train_normalized, y_train)

CPU times: user 1.35 s, sys: 227 ms, total: 1.58 s
Wall time: 886 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
print(model_2.score(X_train_normalized, y_train))
print(roc_auc_score(y_train, model_2.predict_proba(X_train_normalized)[:,1]))

0.91440625
0.8606427353156717


In [54]:
print(model_2.score(X_val_normalized, y_val))
print(roc_auc_score(y_val, model_2.predict_proba(X_val_normalized)[:,1]))

0.914475
0.8629075884886785


In [56]:
model_2.coef_.shape

(1, 200)

In [ ]:
# Ordenamos el valor absoluto de los parámetros de mayor a menor
# Coeficientes ordenados en función de su de mayor importancia
np.argsort(np.abs(model_2.coef_))[0][::-1]

array([ 81, 139,   6,  12,  53, 110,  76,  99, 146, 174,  26,  21,  22,
        80, 190, 166, 165, 198,  13,  34, 169,  44,   2, 148,   0,  40,
       133, 179, 170,  78,  94,   1, 184, 109,  33, 115,  92,  67, 149,
       108, 191, 122, 173, 154,  18,  86, 192, 118, 107, 121, 147,  95,
         9,  75,  35, 164, 177, 197, 172,  36, 127,  89, 123, 155,  91,
       188,  56,  87,  71,  48,   5,  93, 162, 106, 157, 130, 141, 145,
        24, 151,  32, 167, 163, 150, 186, 119,  49,  31, 180,  23, 111,
       195,  90, 131, 125, 137, 114, 199,  43, 116, 135,  52,  58, 128,
        70, 104, 175, 112, 132, 105,  11, 196,  85,  82, 194,  51,  28,
       142,  83,  66, 134, 144, 138,  74,  45, 156,  77,  55,  97, 140,
        20,  54, 193,  57,  88, 178,   8, 102, 113,  62,  15, 143, 159,
       187, 181, 171,  63,  72,  64,  50,  59, 120, 168, 182, 101,  25,
        68,   3,  65, 152,   4,  84,  37,  42, 176,  61, 129,  19, 189,
        69,  47,  60,  16,  27,  29,  79,  73, 158,  96, 160,  4

In [57]:
# Parámetros menores a cero
negative_indexes = np.where(model.coef_<0)[1]
negative_indexes

array([  9,  10,  12,  13,  14,  17,  20,  21,  23,  27,  28,  30,  31,
        33,  34,  36,  38,  39,  41,  42,  43,  44,  45,  50,  54,  56,
        57,  58,  59,  63,  64,  68,  71,  72,  73,  75,  76,  77,  80,
        81,  83,  85,  86,  87,  88,  92,  93,  98, 101, 102, 103, 104,
       107, 109, 113, 114, 115, 116, 120, 121, 122, 123, 127, 129, 131,
       132, 136, 139, 141, 142, 143, 146, 148, 149, 150, 152, 153, 154,
       156, 158, 160, 161, 165, 166, 169, 172, 174, 177, 178, 182, 183,
       186, 188, 192, 193, 194, 197, 198])

In [ ]:
# Parámetros mayores a cero
positive_indexes = np.where(model.coef_>0)[1]
positive_indexes

array([  0,   1,   2,   3,   4,   5,   6,   8,  11,  15,  16,  18,  19,
        22,  24,  25,  26,  29,  30,  32,  35,  37,  38,  40,  46,  47,
        48,  49,  51,  52,  53,  55,  60,  61,  62,  65,  66,  67,  68,
        69,  70,  71,  74,  78,  79,  82,  84,  89,  90,  91,  94,  95,
        96,  97,  99, 100, 103, 105, 106, 108, 110, 111, 112, 117, 118,
       119, 124, 125, 126, 128, 130, 133, 134, 135, 137, 138, 140, 144,
       145, 147, 151, 155, 157, 159, 161, 162, 163, 164, 167, 168, 170,
       171, 173, 175, 176, 179, 180, 181, 184, 187, 189, 190, 191, 195,
       196, 199])

In [ ]:
len(negative_indexes)/len(positive_indexes)

0.8867924528301887

Notar que hay algunos parámetros mayores a cero y otros menores a cero. Como contribuyen en función de su signo?

In [59]:
# Ordenados de menor a mayor
np.argsort(model_2.coef_)[0]

array([ 81, 139,  12,  76, 146, 174,  21,  80, 166, 165, 198,  13,  34,
       169,  44, 148, 109,  33, 115,  92, 149, 108, 122, 154,  86, 192,
       107, 121,   9,  75, 177, 197, 172,  36, 127, 123, 188,  56,  87,
        93, 141, 150, 186,  31,  23, 131, 114,  43, 116,  58, 104, 132,
        85, 194,  28, 142,  83,  45, 156,  77,  20,  54, 193,  57,  88,
       178, 102, 113, 143,  63,  72,  64,  50,  59, 120, 182, 101,  68,
       152,  42, 129,  27,  73, 158, 160,  98,  14, 153,  39, 136, 183,
        41, 103,  10,   7, 185,  38,  17,  30, 161, 100, 117, 126, 124,
        46,  96,  79,  29,  16,  60,  47,  69, 189,  19,  61, 176,  37,
        84,   4,  65,   3,  25, 168, 171, 181, 187, 159,  15,  62,   8,
       140,  97,  55,  74, 138, 144, 134,  66,  51,  82, 196,  11, 105,
       112, 175,  70, 128,  52, 135, 199, 137, 125,  90, 195, 111, 180,
        49, 119, 163, 167,  32, 151,  24, 145, 130, 157, 106, 162,   5,
        48,  71,  91, 155,  89, 164,  35,  95, 147, 118,  18, 17

In [ ]:
# Ordenados de mayor a menor
np.argsort(model_2.coef_)[0][::-1]

array([  6,  53, 110,  99,  26,  22, 190,   2,   0,  40, 133, 179, 170,
        78,  94,   1, 184,  67, 191, 173,  18, 118, 147,  95,  35, 164,
        89, 155,  91,  71,  48,   5, 162, 106, 157, 130, 145,  24, 151,
        32, 167, 163, 119,  49, 180, 111, 195,  90, 125, 137, 199, 135,
        52, 128,  70, 175, 112, 105,  11, 196,  82,  51,  66, 134, 144,
       138,  74,  55,  97, 140,   8,  62,  15, 159, 187, 181, 171, 168,
        25,   3,  65,   4,  84,  37, 176,  61,  19, 189,  69,  47,  60,
        16,  29,  79,  96,  46, 124, 126, 117, 100, 161,  30,  17,  38,
       185,   7,  10, 103,  41, 183, 136,  39, 153,  14,  98, 160, 158,
        73,  27, 129,  42, 152,  68, 101, 182, 120,  59,  50,  64,  72,
        63, 143, 113, 102, 178,  88,  57, 193,  54,  20,  77, 156,  45,
        83, 142,  28, 194,  85, 132, 104,  58, 116,  43, 114, 131,  23,
        31, 186, 150, 141,  93,  87,  56, 188, 123, 127,  36, 172, 197,
       177,  75,   9, 121, 107, 192,  86, 154, 122, 108, 149,  9

In [ ]:
# Calculamos correlación de cada variable con la salida
correlations = []
for i in range(200):
    correlations.append(np.corrcoef(y_train, X_train_normalized[:,i])[1,0])

In [ ]:
np.argsort(np.abs(correlations))[::-1]

array([ 81, 139,  12, 110,   6,  26,  53, 146, 174,  76,  99,  80, 166,
        22, 165,  21, 190, 148,   2,  13,  34, 133,   0, 198, 169, 109,
       179,  44,  40,   1, 115, 149, 170, 184,  78,  94,  92, 108,  67,
       191,  33,  18, 154, 173,  86, 122, 192, 147, 118,   9, 121,  75,
        95, 164, 123,  35,  91,  36, 172, 127, 107, 155,  89, 177, 197,
        56,  93, 188,  87,  71, 162,  48, 157, 106, 141, 145,   5,  32,
       163, 131, 167, 186, 119,  49,  24, 151, 130,  90, 111, 135, 180,
       125,  43,  51, 114, 195, 199,  52,  23, 150,  31, 116,  85,  70,
       137, 105, 128, 104,  58, 112, 196, 132,  11,  28, 175,  66,  82,
       194, 156,  83, 142,  45, 144,  88,  74, 178, 138,  77,   8,  97,
       134,  20,  55, 193,  54,  15, 102,  57, 159, 140, 171,  62, 113,
       187,  63, 143,  64,  25, 181,  50, 120, 168,  59,   3,  72,  65,
        68,  84, 101,  19,   4,  37,  61,  69, 189, 152, 182,  42,  16,
       129, 176,  47,  79, 153,  14, 183,  46, 160,  73,  60,  9

# Feature engineering

In [60]:
# Agregamos 200 columnas con el cuadrado de cada columna normaliado a train
X_train_sq = X_train_normalized**2
means_sq = X_train_sq.mean(axis=0)
stds_sq = X_train_sq.std(axis=0)
X_train_FE = np.append(X_train_normalized, (X_train_sq - means_sq)/stds_sq, axis=1)

# Agregamos el cuadrado a validación normalizando con la media y std obtenida en train
X_val_sq = X_val_normalized**2
X_val_FE = np.append(X_val_normalized, (X_val_normalized**2-means_sq)/stds_sq, axis=1)

In [61]:
model_3 = LogisticRegression(solver='lbfgs')
%time model_3.fit(X_train_FE, y_train)

CPU times: user 2.53 s, sys: 263 ms, total: 2.8 s
Wall time: 1.57 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [62]:
print(model_3.score(X_train_FE, y_train))
print(roc_auc_score(y_train, model_3.predict_proba(X_train_FE)[:,1]))

0.9218625
0.8910807326950416


In [ ]:
print(model_3.score(X_val_FE, y_val))
print(roc_auc_score(y_val, model_3.predict_proba(X_val_FE)[:,1]))

0.9221
0.8927208962331821


In [ ]:
# Coeficientes ordenados en función de su de mayor importancia
np.argsort(np.abs(model_3.coef_))[0][::-1]

array([139,  81,   6,  53,  12,  21,  76, 174, 110,  34, 146,  26, 169,
        99, 165, 281,  22, 166, 190,  78,  33,  40,  13, 170, 149,  80,
       148, 184,  94, 109, 133, 192,  92, 122, 115,  67, 121, 198,   0,
         1, 173, 179,   2, 212,  18, 118, 108, 253, 398, 107, 191, 172,
       177,  44, 154, 310, 164,  35, 197, 226, 188,  36, 339, 299,  75,
       147, 244,   9, 280,  86,  87, 366,  95,  89,  48, 390, 127,  91,
       222, 309,  71, 346, 155, 202,   5, 323, 374, 379,  56, 145, 106,
       162, 123, 206, 221,  32, 186, 141, 200, 150, 348, 151,  31, 365,
       199,  49, 130, 240,  23, 278, 167, 157,  24, 333,  90, 276,  93,
       111, 119, 354, 137, 364, 195, 286, 363, 128, 112, 180, 380,  52,
       104, 114,  58, 131,  70, 355, 116, 125, 135, 295, 370, 233, 391,
       292, 201,  43, 275, 175, 196, 293, 357, 388, 294,  85, 163, 213,
       347,  82, 308, 142, 132,  28, 105,  11, 319, 234, 194, 291,  74,
       205,  97,  77, 140, 315,  45, 322,  20, 369,  55, 267, 25

# Independecia

In [ ]:
accs = []
aucs = []
val_predictions = np.zeros((len(X_val_normalized), 200))
train_predictions = np.zeros((len(X_train_normalized), 200))
for i in tqdm.tqdm(range(200)):
    model_ind = LogisticRegression(solver='lbfgs')
    X_one_train = X_train_normalized[:,i].reshape(-1,1)
    # X_one_train = np.array([X_train_normalized[:,i], X_train_normalized[:,i]**2]).T
    model_ind.fit(X_one_train, y_train)
    X_one_val = X_val_normalized[:,i].reshape(-1,1)
    # X_one_val = np.array([X_val_normalized[:,i], X_val_normalized[:,i]**2]).T
    train_predictions[:, i] = model_ind.predict_proba(X_one_train)[:,1]
    val_predictions[:, i] = model_ind.predict_proba(X_one_val)[:,1]
    acc = model_ind.score(X_one_val, y_val)
    auc = roc_auc_score(y_val, val_predictions[:, i])
    accs.append(acc)
    aucs.append(auc)

100%|██████████| 200/200 [00:20<00:00,  8.22it/s]


In [ ]:
roc_auc_score(y_train, train_predictions.sum(axis=1)) 

0.8692585548756987

In [ ]:
roc_auc_score(y_val, val_predictions.sum(axis=1)) 
# 0.8729094090424484

0.8729094090424484